In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Download VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Yayerad\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [28]:
# Load the existing stock data
dataAAPL = pd.read_csv('../data/yfinance_data/AAPL_historical_data.csv', index_col='Date', parse_dates=True)
dataMSFT = pd.read_csv('../data/yfinance_data/MSFT_historical_data.csv', index_col='Date', parse_dates=True)
dataGOOGL = pd.read_csv('../data/yfinance_data/GOOG_historical_data.csv', index_col='Date', parse_dates=True)

# Load the existing news data
news_data = pd.read_csv('../data/raw_analyst_ratings.csv', parse_dates=['date'])

# Convert to datetime and adjust to UTC
news_data['date'] = pd.to_datetime(news_data['date'], utc=True, format='mixed')
dataAAPL.index = pd.to_datetime(dataAAPL.index, utc=True)
dataMSFT.index = pd.to_datetime(dataMSFT.index, utc=True)
dataGOOGL.index = pd.to_datetime(dataGOOGL.index, utc=True)

# Extract just the date part
news_data['date'] = news_data['date'].dt.date
dataAAPL = dataAAPL.reset_index()
dataMSFT = dataMSFT.reset_index()
dataGOOGL = dataGOOGL.reset_index()

dataAAPL['Date'] = dataAAPL['Date'].dt.date
dataMSFT['Date'] = dataMSFT['Date'].dt.date
dataGOOGL['Date'] = dataGOOGL['Date'].dt.date


In [29]:
# Calculate daily percentage changes
dataAAPL['Daily_Return'] = dataAAPL['Close'].pct_change() * 100  # Multiply by 100 to get percentage
dataMSFT['Daily_Return'] = dataMSFT['Close'].pct_change() * 100  # Multiply by 100 to get percentage
dataGOOGL['Daily_Return'] = dataGOOGL['Close'].pct_change() * 100  # Multiply by 100 to get percentage


In [30]:
# Merge on the date
merged_data_AAPL = pd.merge(news_data, dataAAPL, left_on='date', right_on='Date', how='left')
merged_data_MSFT = pd.merge(news_data, dataMSFT, left_on='date', right_on='Date', how='left')
merged_data_GOOGL = pd.merge(news_data, dataGOOGL, left_on='date', right_on='Date', how='left')

# Handle non-trading days by filling missing stock data with the closest trading day
merged_data_AAPL = merged_data_AAPL.ffill()  # Forward fill
merged_data_MSFT = merged_data_MSFT.ffill()  # Forward fill
merged_data_GOOGL = merged_data_GOOGL.ffill()  # Forward fill

print("Merged Data Sample for AAPL:")
print(merged_data_AAPL.head())


Merged Data Sample for AAPL:
   Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzinga Insights   
1  https://www.benzinga.com/news/20/06/16170189/s...  Benzinga Insights   
2  https://www.benzinga.com/news/20/05/16103463/7...         Lisa Levin   
3  https://www.benzinga.com/news/20/05/16095921/4...         Lisa Levin   
4  https://www.benzinga.com/news/20/05/16095304/b...         Vick Meyer   

         date stock        Date       Open       High        Low      Close  \
0  2020-06-05     A  2020-06-05 

In [31]:
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Perform sentiment analysis on headlines
def get_sentiment_score(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']  # Compound score

# Apply sentiment analysis
merged_data_AAPL['sentiment_score'] = merged_data_AAPL['headline'].apply(get_sentiment_score)
merged_data_MSFT['sentiment_score'] = merged_data_MSFT['headline'].apply(get_sentiment_score)
merged_data_GOOGL['sentiment_score'] = merged_data_GOOGL['headline'].apply(get_sentiment_score)

# Display the merged DataFrame with sentiment scores
print("Merged Data with Sentiment Scores for AAPL:")
print(merged_data_AAPL.head())


Merged Data with Sentiment Scores for AAPL:
   Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzinga Insights   
1  https://www.benzinga.com/news/20/06/16170189/s...  Benzinga Insights   
2  https://www.benzinga.com/news/20/05/16103463/7...         Lisa Levin   
3  https://www.benzinga.com/news/20/05/16095921/4...         Lisa Levin   
4  https://www.benzinga.com/news/20/05/16095304/b...         Vick Meyer   

         date stock        Date       Open       High        Low      Close  \
0  2020-06-05    

In [32]:
# Aggregate sentiment scores by date (compute the average)
daily_sentiments_AAPL = merged_data_AAPL.groupby('date')['sentiment_score'].mean().reset_index()
daily_sentiments_MSFT = merged_data_MSFT.groupby('date')['sentiment_score'].mean().reset_index()
daily_sentiments_GOOGL = merged_data_GOOGL.groupby('date')['sentiment_score'].mean().reset_index()

# Display the daily sentiment scores
print("Daily Sentiments for AAPL:")
print(daily_sentiments_AAPL.head())


Daily Sentiments for AAPL:
         date  sentiment_score
0  2009-02-14          0.22630
1  2009-04-27          0.00000
2  2009-04-29          0.00000
3  2009-05-22          0.00000
4  2009-05-27          0.75105


In [33]:
# Ensure stock data contains 'Date' and 'Daily_Return' columns
dataAAPL = dataAAPL.rename(columns={'Date': 'date'})
dataMSFT = dataMSFT.rename(columns={'Date': 'date'})
dataGOOGL = dataGOOGL.rename(columns={'Date': 'date'})

dataAAPL = dataAAPL[['date', 'Daily_Return']]  # Ensure only relevant columns are included
dataMSFT = dataMSFT[['date', 'Daily_Return']]
dataGOOGL = dataGOOGL[['date', 'Daily_Return']]

# Merge aggregated sentiment scores with stock data
merged_data_AAPL = pd.merge(daily_sentiments_AAPL, dataAAPL, on='date', how='left')
merged_data_MSFT = pd.merge(daily_sentiments_MSFT, dataMSFT, on='date', how='left')
merged_data_GOOGL = pd.merge(daily_sentiments_GOOGL, dataGOOGL, on='date', how='left')

# Display results
print("Merged Data with Aggregated Sentiments and Daily Returns for AAPL:")
print(merged_data_AAPL.head())


Merged Data with Aggregated Sentiments and Daily Returns for AAPL:
         date  sentiment_score  Daily_Return
0  2009-02-14          0.22630           NaN
1  2009-04-27          0.00000      0.669889
2  2009-04-29          0.00000      1.000808
3  2009-05-22          0.00000     -1.352874
4  2009-05-27          0.75105      1.735759


In [34]:
# Compute correlation between sentiment scores and daily returns
correlation_AAPL = merged_data_AAPL[['sentiment_score', 'Daily_Return']].corr().iloc[0, 1]
correlation_MSFT = merged_data_MSFT[['sentiment_score', 'Daily_Return']].corr().iloc[0, 1]
correlation_GOOGL = merged_data_GOOGL[['sentiment_score', 'Daily_Return']].corr().iloc[0, 1]

# Display the correlation results
print(f"Correlation between Sentiment Scores and Daily Returns for AAPL: {correlation_AAPL:.2f}")
print(f"Correlation between Sentiment Scores and Daily Returns for MSFT: {correlation_MSFT:.2f}")
print(f"Correlation between Sentiment Scores and Daily Returns for GOOGL: {correlation_GOOGL:.2f}")


Correlation between Sentiment Scores and Daily Returns for AAPL: 0.15
Correlation between Sentiment Scores and Daily Returns for MSFT: 0.12
Correlation between Sentiment Scores and Daily Returns for GOOGL: 0.13
